In [1]:
import numpy as np
import json 
import scanpy as sc
from collections import OrderedDict
import scipy 

#spectra imports 
from spectra import spectra as spc
from spectra import spectra_util as util
from spectra import K_est as kst

## Load gene sets

In [2]:
with open("annotations_2.json", "rb") as file:
    annotations = json.load(file)

## Load data 

In [4]:
adata = sc.read_h5ad(PATH_TO_DATA + "Bassez_10k_subset_scran.h5ad")

## K estimation

In [16]:
# don't rerun this unless necessary
L = K_est.estimate_L(adata, attribute = "annotation_SPADE_1", highly_variable = True)

In [18]:
L

{'global': 20, 'B': 31, 'M': 15, 'Neutrophil': 3, 'TNK': 13}

## Fit SPECTRA model 

> don't rerun this

In [ ]:
model = spc.est_spectra(adata = adata, L = L, gene_set_dictionary = annotations, use_highly_variable = True, cell_type_key = "annotation_SPADE_1", use_weights = True, lam = 0.1, delta=0.001,kappa = 0.00001, rho = 0.00001, use_cell_types = True, n_top_vals = 25)

 23%|██▎       | 2301/10000 [6:06:03<20:07:31,  9.41s/it]

In [ ]:
model.save("example_model")